## Importing Libraries

In [1]:
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import model
import process

%load_ext autoreload
%autoreload 2

c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

In [ ]:
# model = model_.model_structure()

## Load weights from pretrained model

In [ ]:
# # Load weights from csv files (which was exported from Openface torch model)
# weights = utils.weights
# weights_dict = utils.load_weights()

# # Set layer weights of the model
# for name in weights:
#     if model.get_layer(name) != None:
#         model.get_layer(name).set_weights(weights_dict[name])
#     elif model.get_layer(name) != None:
#         model.get_layer(name).set_weights(weights_dict[name])

In [ ]:
# model.save_weights('check')

In [2]:
model = model.model_structure()

In [3]:
model.load_weights('check')

## Capturing faces

## - Webcam

In [ ]:
def face_capture(userName):
    cam = cv2.VideoCapture(0)

    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    count = 0
    while(True):
        ret, img = cam.read()
        cv2.imshow("image",img)
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
            count += 1
            # Save the captured image into the datasets folder
            if count == 10:
                cv2.imwrite(f"images/{userName}" + ".jpg", img[y1:y2,x1:x2])
        k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 30: # Take face sample and stop video
             break
    cam.release()
    cv2.destroyAllWindows()

## - ImagePath

In [ ]:
def face_capture_image(userName,imgPath):
    imgPath = f"addUser/{imgPath}"
    img = cv2.imread(imgPath)
    WIDTH = 600
    img = imutils.resize(img,width=WIDTH)

    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    faces = face_detector.detectMultiScale(img,1.3,5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img,(x1,y1),(x2,y2),(255,255,255),2)

        cv2.imwrite(f"images/{userName}"+".jpg",img[y1:y2,x1:x2])
        print(f"User Added {userName}")
        cv2.destroyAllWindows()

## 1. Capture faces

In [ ]:
import matplotlib.pyplot as plt
import os

In [ ]:
user = input("Enter name of the user ")
z = os.listdir("images/")
if f"{user}.jpg" not in z:
    face_capture(user)
else:
    print("User already present in the database")
    option = int(input("\nChoose option\n1.Update\n2.Cancel\n\n"))
    if option == 1:
        face_capture(user)

# path = f'images/{user}.jpg'
# im = plt.imread(path)
# plt.xticks([])
# plt.yticks([])
# plt.title(user)
# plt.imshow(im)

In [ ]:
import cv2
import os
import imutils

user = input("Enter name of the user ")
z = os.listdir("images/")
if f"{user}.jpg" not in z:
    print("\n#### Put your image in the addUser directory####\n")
    imageName = input("Enter the name of the image with type\nExample\n- image.jpg\n\nImage Name: ")
    face_capture_image(userName=user,imgPath=imageName)
else:
    print("User already present in the database")
    option = int(input("\nChoose option\n1.Update\n2.Cancel\n\n"))
    if option == 1:
        face_capture_image(userName=user,imgPath=imageName)

## 2. Run the model

In [14]:
input_embeddings = process.create_input_image_embeddings(model)
# process.recognize_faces_in_cam(input_embeddings,model)

## Testing and Running

In [21]:
import pickle
def ini_user_database():
    # check for existing database
    if os.path.exists('database/input_embeddings.pickle'):
        with open('database/input_embeddings.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        # make a new one
        # we use a dict for keeping track of mapping of each person with his/her face encoding
        user_db = {}
        # create the directory for saving the db pickle file
        os.makedirs('database')
        with open('database/input_embeddings.pickle', 'wb') as handle:
            pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)   
    return user_db

input_embeddings = ini_user_database()

{'ankit': array([[-0.04535592,  0.11877114,  0.09604801, -0.02746559,  0.00655375,
          0.2664417 , -0.07279567,  0.02875434, -0.03558714, -0.05765909,
          0.00506036, -0.07230225, -0.00255757, -0.12326233,  0.11683905,
         -0.0601402 , -0.10549192, -0.04574217, -0.04436597, -0.00433365,
          0.05151786, -0.06399747,  0.07662596,  0.05259002,  0.08408903,
         -0.06714708, -0.09242869, -0.08870233,  0.04782192, -0.15571295,
          0.28809378, -0.07690366, -0.06725828,  0.07135818,  0.05038142,
          0.13815278, -0.04084692,  0.01621393,  0.01604621, -0.00255266,
          0.12451801, -0.10823712, -0.05615313,  0.020857  , -0.06483253,
         -0.09724224,  0.07040034,  0.06448827, -0.13595532,  0.03986988,
         -0.10748555,  0.01415718,  0.0183325 , -0.05012123, -0.02602401,
          0.04800905, -0.05382023,  0.11005878, -0.03181394, -0.15680717,
         -0.18972412,  0.11052765,  0.1222297 , -0.1917675 ,  0.08384909,
          0.09970597,  0.0748

- 1. Realtime Face Detetction

In [17]:
# with open('input_embeddings.pickle','wb') as handle:
#     pickle.dump(input_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
# with open('input_embeddings.pickle', 'rb') as handle:
#     input_embeddings = pickle.load(handle)  

In [22]:
process.recognize_faces_in_cam(input_embeddings,model)